In [ ]:
import numpy as np
import h5py
import shutil

In [ ]:
def get_spectre_points_per_block(spectre_file):
    """Reads a file produced by spectre's ExportCoordinates
    and returns two lists: i) the number of points in each block
    and ii) the names of the blocks."""
    observation_id = list(spectre_file['element_data.vol'].keys())[0]
    coords_dict = dict(spectre_file['element_data.vol'][observation_id])
    components = ['InertialCoordinates_x', 'InertialCoordinates_y', 'InertialCoordinates_z']
    
    blocks = list(coords_dict.keys())[:]
    
    points_per_block = []
    for block in blocks:
        points_per_block.append(len(coords_dict[block][components[0]]))
    return np.array(points_per_block), np.array(blocks)

def get_spec_points(spectre_file):
    """Converts points from a file produced by spectre's ExportCoordinates
    to a flat list of points, suitable for SpEC to read in (e.g., 
    for interpolating data onto those points)"""
    observation_id = list(spectre_file['element_data.vol'].keys())[0]
    coords_dict = dict(spectre_file['element_data.vol'][observation_id])

    blocks = list(coords_dict.keys())[:]
    components = ['InertialCoordinates_x', 'InertialCoordinates_y', 'InertialCoordinates_z']
    dim = len(components)
    
    coords = []
    for component in components:
        coords.append([])
        
    for block in blocks:
        for i,component in enumerate(components):
            coords[i].append(coords_dict[block][component])
    return np.transpose(np.array([np.concatenate(x) for x in coords]))

def get_spectre_values_from_spec_values(array_1D, spectre_points_file):
    """Take a 1D array of values, listed point-by-point (as returned by spec),
    and return a dictionary, keyed by block name, of the values at each spectre point.
    spectre_points_file is a file containing the structured spectre points, e.g., 
    the file output by ExportCoordinates."""
    points_per_block, blocks = get_spectre_points_per_block(file)
    points = get_spec_points(file)
    
    spectre_values = {}
    for i,block in enumerate(blocks):
        start = np.sum(points_per_block[:i])
        end = start + points_per_block[i]
        spectre_values[block] = array_1D[start:end]
    return spectre_values

In [ ]:
%%time

# Convert spectre points to spec and write the results to a file
file = h5py.File("VolumeData0.h5", 'r')
points_per_block, blocks = get_spectre_points_per_block(file)
points = get_spec_points(file)
file.close()
print(points.shape)
np.savetxt("PointsList.txt", points)

In [ ]:
%%time

# Convert points in spectre format to spec
file = h5py.File("VolumeData0.h5", 'r')
points_per_block, blocks = get_spectre_points_per_block(file)
points = get_spec_points(file)

# Treat x,y,z as scalars in spec format
# Convert to spectre format and make sure you recover the 
# original coordinates
test_coords = np.array([get_spectre_values_from_spec_values(points[:,0], file), 
                        get_spectre_values_from_spec_values(points[:,1], file),
                        get_spectre_values_from_spec_values(points[:,2], file)])

observation_id = list(file['element_data.vol'].keys())[0]
components = ['InertialCoordinates_x', 'InertialCoordinates_y', 'InertialCoordinates_z']
all_points_the_same = []
for block in blocks:
    for i,component in enumerate(components):
        all_the_same_this_block = np.all(np.array(test_coords[i][block]) 
                                         == np.array(file['element_data.vol'][observation_id][block][component]))
        all_points_the_same = all_points_the_same and all_the_same_this_block

print("All x,y,z coordinates are the same: " + str(np.all(all_points_the_same)))
file.close()

In [ ]:
# Test reading in some interpolated data
lapse = np.genfromtxt("spec/Lev1/Run/Lapse.dat")
file = h5py.File("VolumeData0.h5", 'r')
points_per_block, blocks = get_spectre_points_per_block(file)
observation_id = list(file['element_data.vol'].keys())[0]
points = get_spec_points(file)
lapse_spectre = get_spectre_values_from_spec_values(lapse, file)
file.close()

# Test writing interpolated data into an H5 file
shutil.copyfile("VolumeData0.h5", "ID_VolumeData0.h5")
outfile = h5py.File("ID_VolumeData0.h5", 'a')
for block in blocks:
    outfile['element_data.vol'][observation_id][block]["Lapse.dat"] = lapse_spectre[block]
outfile.close()